In [4]:
%pip install --upgrade llama-index        # core
%pip install --upgrade llama-index-llms-google-genai  # Google / Gemini LLM integration
%pip install --upgrade llama-index-embeddings-google-genai  # embeddings via Google GenAI
%pip install --upgrade google-generativeai  # underlying Google SDK
%pip install --upgrade pinecone-client      # if using Pinecone
    


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from pinecone import Pinecone
from llama_index.vector_stores.pinecone import PineconeVectorStore
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader,Settings, get_response_synthesizer
from llama_index.llms.google_genai import GoogleGenAI
from llama_index.embeddings.google_genai import GoogleGenAIEmbedding
from llama_index.core.retrievers import QueryFusionRetriever
from llama_index.core.query_engine import RetrieverQueryEngine

In [26]:
# Force Google models globally BEFORE anything else (prevents OpenAI
os.environ.pop("OPENAI_API_KEY", None) # ensure no accidental

Settings.embed_model = GoogleGenAIEmbedding(model_name="models/text-embedding-004",api_key="")
Settings.llm = GoogleGenAI(
    model="gemini-2.5-flash",
    api_key="",
    use_async=True  
)


2025-11-28 22:54:26,580 - INFO - HTTP Request: GET https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash "HTTP/1.1 200 OK"


In [27]:
# =============================================
# Data layer existing index with 004 emebeding model
# =============================================

INDEX_NAME = "coffeeindex"
pc = Pinecone(api_key="")
pc_index = pc.Index(INDEX_NAME)
vstore = PineconeVectorStore(pinecone_index=pc_index)

In [28]:
os.getcwd()

'/Users/devenderswami'

In [29]:
# Local PDFs (NO Pinecone upsert; kept in-memory)
docs = SimpleDirectoryReader("./GenAI/GenAI-NoteBooks/coffee_pages").load_data()

In [30]:
# Build indexes (use Settings.embed_model)
vector_idx = VectorStoreIndex.from_vector_store(vstore) #Pinecone-backed
local_idx = VectorStoreIndex.from_documents(docs) #in-memory for PDF

2025-11-28 22:54:31,154 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/text-embedding-004:batchEmbedContents "HTTP/1.1 200 OK"
2025-11-28 22:54:31,603 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/text-embedding-004:batchEmbedContents "HTTP/1.1 200 OK"
2025-11-28 22:54:32,091 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/text-embedding-004:batchEmbedContents "HTTP/1.1 200 OK"


In [31]:
# ============================ Query layer
#============================ (Optional for now)
# Auto-fuse Pinecone + PDFs; no query expansion (no hidden LLM calls in retrieval)
pinecone_ret = vector_idx.as_retriever(similarity_top_k=5)
pdf_ret = local_idx.as_retriever(similarity_top_k=5)
hybrid = QueryFusionRetriever(
 retrievers=[pinecone_ret, pdf_ret],
 similarity_top_k=5,
 mode="reciprocal_rerank",
 num_queries=1,
 use_async=False,
)

In [32]:
qe = RetrieverQueryEngine.from_args(
 retriever=hybrid,
 response_synthesizer=get_response_synthesizer(llm=Settings.llm,
response_mode="compact")
)

In [34]:
import nest_asyncio
nest_asyncio.apply()

In [35]:
print(qe.query("Using both PDFs and the vector KB, explain desi brewing in 3 bullets."))

2025-11-28 22:57:25,148 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/text-embedding-004:batchEmbedContents "HTTP/1.1 200 OK"
2025-11-28 22:57:26,300 - INFO - AFC is enabled with max remote calls: 10.
2025-11-28 22:57:33,165 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent "HTTP/1.1 200 OK"


*   A prominent method involves South Indian filter coffee, which uses a metal filter, dark roast coffee (frequently blended with chicory), and hot milk, traditionally aerated in a tumbler and dabara.
*   Many Indian coffee preparations, such as Masala Coffee and Cold Brew with Indian Spices, incorporate classic Indian spices like cardamom, cinnamon, and clove, either infused directly or through syrups.
*   These brewing styles are deeply embedded in Indian culture, serving as symbols of hospitality and morning rituals, and may include traditional sweeteners like jaggery.
